### 2325165 rows × 15 columns

#### Random forest

Jaman dulu pas kolomnya sedikit:
```
X=df[['productGroup','quantity','price','rrp','voucherAmount','deviceID']].fillna(0).values
error=924799
```

Jaman sekarang

error=41429

In [1]:
label='returnQuantity'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.cross_validation import train_test_split
from datetime import datetime
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from __future__ import division
%matplotlib inline
from sklearn.externals import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def dmc_error(returnQuantity, prediction):
    #∑ |returnQuantity𝑖 − prediction𝑖 |
    return np.sum(np.abs( np.subtract(returnQuantity, prediction) ))

def stratify(labels):
    label_groups = {}
    for index in range(0,len(labels)):
        label = labels.iloc[index]
        # label = labels[index]
        if not (label in label_groups):
            label_groups[label] = [index]
        else:
            label_groups[label].append(index)
    return label_groups

def stratify_nparray(labels):
    label_groups = {}
    for index in range(0,len(labels)):
        # label = labels.iloc[index]
        label = labels[index]
        if not (label in label_groups):
            label_groups[label] = [index]
        else:
            label_groups[label].append(index)
    return label_groups

def sample(data, target, proportion=0.5):
    """
    @param: data is data/attributes as DataFrame objects
    @param: target is data/attributes as DataFrame objects
    @param: proportion is percentage of sample in float
    """
    samples = []
    groups = stratify(target)
    for label in groups:
        group = groups[label]
        size = len(group)
        sample_size = int(size*proportion)
        if (sample_size < 50):
            sample_size = size
        samples.extend(random.sample(group, sample_size))
    
    #return data[samples], target[samples]
    return data.ix[samples], target.ix[samples]

def sample_nparray(data, target, proportion=0.5):
    samples = []
    groups = stratify_nparray(target)
    for label in groups:
        group = groups[label]
        size = len(group)
        samples.extend(random.sample(group, int(size*proportion)))
    
    return data[samples], target[samples]
    #return data.ix[samples], target.ix[samples]    

In [5]:
#df=pd.read_csv('orders_train.txt', sep=';'); df

,orderID,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,voucherID,voucherAmount,customerID,deviceID,paymentMethod,returnQuantity
0,a1000001,2014-01-01,i1000382,1972,44,3,1,10.00,29.99,0,0,c1010575,2,BPRG,0
1,a1000001,2014-01-01,i1000550,3854,44,3,1,20.00,39.99,0,0,c1010575,2,BPRG,0
2,a1000002,2014-01-01,i1001991,2974,38,8,1,35.00,49.99,0,0,c1045905,4,BPRG,0
3,a1000002,2014-01-01,i1001999,1992,38,8,1,49.99,49.99,0,0,c1045905,4,BPRG,1
4,a1000003,2014-01-01,i1001942,1968,42,8,1,10.00,35.99,0,0,c1089295,2,PAYPALVC,0
5,a1000003,2014-01-01,i1001942,1972,42,8,1,10.00,35.99,0,0,c1089295,2,PAYPALVC,0
6,a1000003,2014-01-01,i1001974,1001,42,8,1,25.00,39.99,0,0,c1089295,2,PAYPALVC,0
7,a1000003,2014-01-01,i1001976,3976,44,8,1,15.00,39.99,0,0,c1089295,2,PAYPALVC,0
8,a1000003,2014-01-01,i1002392,2493,42,13,0,0.00,59.99,0,0,c1089295,2,PAYPALVC,0
9,a1000004,2014-01-01,i1002457,1001,42,14,1,89.99,89.99,0,0,c1050116,3,BPRG,1


In [2]:
#joblib.dump(df, 'df.pkl') 
orig_df=joblib.load('df.pkl')
df=orig_df.copy()

In [3]:
df['mmdd']=orig_df.orderDate.str[5:];

In [4]:
#Konversi kategori/object ke numerik

#Cari kolom yang tipenya object, bukan integer maupun float
object_columns=df.loc[:, df.dtypes == object].columns
for col in object_columns:
    print col
    le=LabelEncoder()
    # Konversi deh
    df[col]=le.fit_transform(df[col])
    
#df['price_delta']=df.rrp-df.price;# df.price_delta.head()

orderID
orderDate
articleID
sizeCode
voucherID
customerID
paymentMethod
mmdd


In [59]:
order=df[['orderID', 'price']].groupby('orderID').sum()
order.columns=['total_order']
df=df.set_index('orderID')
df=df.join(order)
df=df.reset_index()

,orderID,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,voucherID,voucherAmount,deviceID,paymentMethod,returnQuantity,mmdd,total_order
0,a1000001,2014-01-01,i1000382,1972,44,3,1,10.00,29.99,0,0,2,BPRG,0,01-01,30.00
1,a1000001,2014-01-01,i1000550,3854,44,3,1,20.00,39.99,0,0,2,BPRG,0,01-01,30.00
2,a1000002,2014-01-01,i1001991,2974,38,8,1,35.00,49.99,0,0,4,BPRG,0,01-01,84.99
3,a1000002,2014-01-01,i1001999,1992,38,8,1,49.99,49.99,0,0,4,BPRG,1,01-01,84.99
4,a1000003,2014-01-01,i1001942,1968,42,8,1,10.00,35.99,0,0,2,PAYPALVC,0,01-01,60.00
5,a1000003,2014-01-01,i1001942,1972,42,8,1,10.00,35.99,0,0,2,PAYPALVC,0,01-01,60.00
6,a1000003,2014-01-01,i1001974,1001,42,8,1,25.00,39.99,0,0,2,PAYPALVC,0,01-01,60.00
7,a1000003,2014-01-01,i1001976,3976,44,8,1,15.00,39.99,0,0,2,PAYPALVC,0,01-01,60.00
8,a1000003,2014-01-01,i1002392,2493,42,13,0,0.00,59.99,0,0,2,PAYPALVC,0,01-01,60.00
9,a1000004,2014-01-01,i1002457,1001,42,14,1,89.99,89.99,0,0,3,BPRG,1,01-01,89.99


In [92]:
df=df.reset_index()
max_order=df[['customerID','total_order']].groupby('customerID').max()
max_order.columns=['max_order']
max_order
df=df.set_index('customerID')
df=df.join(max_order)
df=df.reset_index()
df

,customerID,index,orderID,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,voucherID,voucherAmount,deviceID,paymentMethod,returnQuantity,mmdd,total_order,max_order
0,c1000001,4157,a1001465,2014-01-03,i1003077,3999,I,17,1,27.99,27.99,0,0,2,BPRG,0,01-03,27.99,719.85
1,c1000001,14749,a1004977,2014-01-07,i1003082,1999,I,17,1,25.99,25.99,0,0,2,BPRG,1,01-07,25.99,719.85
2,c1000001,37290,a1012517,2014-01-15,i1001229,1993,38,4,1,39.99,39.99,0,0,2,BPRG,0,01-15,39.99,719.85
3,c1000001,40599,a1013536,2014-01-16,i1002531,1995,38,15,1,59.99,59.99,0,0,2,BPRG,1,01-16,119.98,719.85
4,c1000001,40600,a1013536,2014-01-16,i1002531,1999,40,15,1,59.99,59.99,0,0,2,BPRG,1,01-16,119.98,719.85
5,c1000001,64800,a1021517,2014-01-24,i1000357,2996,40,3,1,39.99,39.99,v1000086,10,2,BPRG,1,01-24,65.98,719.85
6,c1000001,64801,a1021517,2014-01-24,i1003093,1993,I,17,1,25.99,25.99,v1000086,10,2,BPRG,0,01-24,65.98,719.85
7,c1000001,69192,a1022902,2014-01-26,i1001387,1993,40,5,1,39.99,39.99,0,0,2,BPRG,1,01-26,59.98,719.85
8,c1000001,69193,a1022902,2014-01-26,i1003098,3999,I,17,1,19.99,19.99,0,0,2,BPRG,0,01-26,59.98,719.85
9,c1000001,87150,a1028928,2014-02-02,i1000744,3002,42,3,1,12.99,12.99,v1000111,0,2,BPRG,1,02-02,143.91,719.85


In [105]:
df=df.reset_index()
mean_article=df[['articleID','total_order']].groupby('articleID').mean()
mean_article.columns=['mean_article']
mean_article
df=df.set_index('articleID')
df=df.join(mean_article)
df=df.reset_index()
df

,orderID,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,voucherID,voucherAmount,customerID,deviceID,paymentMethod,mmdd,subtotal
0,0,0,346,1972,16,3,1,10,29,6,0,9962,2,2,0,10
1,0,0,346,1972,16,3,1,10,29,6,0,9962,2,2,0,10
2,0,0,346,1972,16,3,1,10,29,6,0,9962,2,2,0,10
3,0,0,346,1972,16,3,1,10,29,6,0,9962,2,2,0,10
4,0,0,346,1972,16,3,1,10,29,6,0,9962,2,2,0,10
5,0,0,346,1972,16,3,1,10,29,6,0,9962,2,2,0,10
6,0,0,346,1972,16,3,1,10,29,6,0,9962,2,2,0,10
7,0,0,346,1972,16,3,1,10,29,6,0,9962,2,2,0,10
8,0,0,346,1972,16,3,1,10,29,6,0,9962,2,2,0,10
9,0,0,346,1972,16,3,1,10,29,6,0,9962,2,2,0,10


In [6]:
label='returnQuantity'
df2=df.dropna()

Xdf=df2.drop(label, axis=1)
Xdf.price=Xdf.price.astype(np.int) # iseng
Xdf.rrp=Xdf.rrp.astype(np.int) # iseng
Xdf
X=Xdf.values
y=df2.returnQuantity.values

### TODO:

- 'colorCode', 'sizeCode' ada huruf I
- split orderDate
- dan lain-lain https://docs.google.com/document/d/1TIXlB9brKg4FER7-6mkQoc07QKGlUtJ4u3UPnYS7g48/edit?ts=57052864

### DROPNA:
```
[('quantity', 0.00730203228970894),
 ('productGroup', 0.010106733663168398),
 ('voucherAmount', 0.015111438226865281),
 ('voucherID', 0.02618833592815939),
 ('paymentMethod', 0.027169190392411509),
 ('rrp', 0.03064330128253916),
 ('deviceID', 0.036043408326550078),
 ('price', 0.041592256668915265),
 ('sizeCode', 0.082797170726113728),
 ('articleID', 0.083584520765206768),
 ('colorCode', 0.089689927276374551),
 ('orderDate', 0.15055960075881508),
 ('orderID', 0.19472601594609326),
 ('customerID', 0.20448606774907857)]
 ```

In [7]:
rfc=RandomForestClassifier(n_jobs=-1)
rfc.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [8]:
from itertools import izip as zip
pair=zip(Xdf.columns, rfc.feature_importances_)
t1=[i for i in pair]; t1
#
from operator import itemgetter
sorted(t1, key=itemgetter(1))

[('quantity', 0.0098347944861536425),
 ('productGroup', 0.010120772873067532),
 ('voucherAmount', 0.013960200008256055),
 ('voucherID', 0.023311490696811044),
 ('paymentMethod', 0.026819967873882265),
 ('rrp', 0.027076705126217765),
 ('deviceID', 0.034029610409893787),
 ('price', 0.041161703895329957),
 ('articleID', 0.083137411906276812),
 ('sizeCode', 0.084620678335968438),
 ('colorCode', 0.089766396810847623),
 ('orderDate', 0.10623509074468467),
 ('mmdd', 0.10670261586215306),
 ('orderID', 0.16503662557438586),
 ('customerID', 0.17818593539607153)]

In [6]:
cv_scores=cross_val_score(rfc, X, y); cv_scores

array([ 0.59122232,  0.58650805,  0.58661408])

In [9]:
y_pred=rfc.predict(X)

In [10]:
dmc_error(df2.returnQuantity.values, np.array(y_pred))

42664

In [23]:
confusion_matrix(df2.returnQuantity.values, np.array(y_pred))

array([[1106308,   10485,      12,       0,       0,       0],
       [  30579, 1173412,      11,       2,       0,       1],
       [     86,     102,    3561,       0,       0,       0],
       [      8,       1,       0,     183,       0,       0],
       [      3,       0,       0,       0,      32,       0],
       [      2,       0,       0,       0,       0,      26]])

In [1]:
df

NameError: name 'df' is not defined

In [ ]:
for train, test in KFold(len(y), n_folds):
    model.fit(X,y)
    y=model.predict(X)
    return 

In [ ]:
df.rrp